In [1]:
import os
import yaml
import openai
from openai import OpenAI

with open("../conf/service.dev.yaml", 'r') as f:
    configs = yaml.safe_load(f)
os.environ['OPENAI_API_KEY'] = configs['openai']['api_key']

client = OpenAI()

# 1. Define a function

In [2]:
import json

def get_current_weather(location, unit='fahrenheit'):
    """Get the current weather in a given location"""
    weather_info = {
        'location': location,
        'temperature': '72',
        'unit': unit,
        'forecast': ['sunny', 'windy']
    }
    return json.dumps(weather_info)

In [3]:
functions = [
    {
        'name': 'get_current_weather',
        'description': 'Get the current weather in a given location',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': "The city and state, e.g. San Francisco, CA"
                },
                'unit': {
                    'type': 'string',
                    'enum': ['celsius', 'fahrenheit']
                }
            },
            'required': ['location']
        }
    }
]

# 2. Relevant message

In [4]:
messages = [
    {
        'role': 'user',
        'content': "What's the weather like in Boston?"
    }
]

In [5]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions
)

In [6]:
response

ChatCompletion(id='chatcmpl-8p8oFBSYnutbf3gbQZmPlRir5bcoa', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1707199283, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100))

In [7]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [8]:
function_call_args_str  = response_message.function_call.arguments
function_call_args_dict = eval(function_call_args_str)  # or json.loads()
assert isinstance(function_call_args_str, str) and isinstance(function_call_args_dict, dict)
function_call_args_dict

{'location': 'Boston, MA'}

In [9]:
args = function_call_args_dict
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

# 3. Unrelevant message

In [10]:
messages = [
    {
        'role': 'user',
        'content': "hi!"
    }
]

In [11]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions
)
print(response)
response.choices[0].message

ChatCompletion(id='chatcmpl-8p8oFGjDq1SUVEKLDXeyWBgfbV3OE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1707199283, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86))


ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)

# 4. Forcing to use function calling

In [12]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions,
    function_call={'name': 'get_current_weather'}
)
print(response)
response.choices[0].message

ChatCompletion(id='chatcmpl-8p8oGhwhD9by55CkteCU9rIZ771MH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"location": "San Francisco, CA"\n}', name='get_current_weather'), tool_calls=None))], created=1707199284, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=83, total_tokens=94))


ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"location": "San Francisco, CA"\n}', name='get_current_weather'), tool_calls=None)

# 5. Forcing not to use function calling

In [13]:
messages = [
    {
        'role': 'user',
        'content': "What's the weather like in Boston?"
    }
]

In [14]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions,
    function_call='none',
)
print(response)
response.choices[0].message

ChatCompletion(id='chatcmpl-8p8oHzhK6tttyaFrs9onJAm0mDffp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Please give me a moment. I will check the current weather in Boston for you.', role='assistant', function_call=None, tool_calls=None))], created=1707199285, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=83, total_tokens=100))


ChatCompletionMessage(content='Please give me a moment. I will check the current weather in Boston for you.', role='assistant', function_call=None, tool_calls=None)

# 6. Prompt size

In [15]:
messages = [
    {
        'role': 'user',
        'content': "What's the weather like in Boston?"
    }
]

In [16]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages
)
response.usage

CompletionUsage(completion_tokens=55, prompt_tokens=15, total_tokens=70)

In [17]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions
)
response.usage

CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100)

In [18]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions,
    function_call='none'
)
response.usage

CompletionUsage(completion_tokens=11, prompt_tokens=83, total_tokens=94)

In [19]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions,
    function_call={'name': 'get_current_weather'}
)
response.usage

CompletionUsage(completion_tokens=11, prompt_tokens=89, total_tokens=100)

# 7. NLG using the function calling result

In [20]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston?"}]

In [21]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [22]:
messages.append(response_message)
messages

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)]

In [23]:
args = eval(response_message.function_call.arguments)
args

{'location': 'Boston, MA'}

In [24]:
observation = get_current_weather(**args)
observation

'{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [25]:
messages.append(
    {
        'role': 'function',
        'name': 'get_current_weather',
        'content': observation
    }
)

In [26]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None),
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]

In [27]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0613',
    messages=messages
)
print(response)
response.choices[0].message.content

ChatCompletion(id='chatcmpl-8p8oNbbZuWGXg9WH9A1WVItyLMH8N', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Boston is sunny and windy with a temperature of 72 degrees Fahrenheit.', role='assistant', function_call=None, tool_calls=None))], created=1707199291, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=74, total_tokens=92))


'The current weather in Boston is sunny and windy with a temperature of 72 degrees Fahrenheit.'